In [21]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import linear_model

In [22]:
loans = pd.read_csv("../data/lending-club-data.csv")

In [23]:
loans.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


In [24]:
features = list(loans)
print features

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans', 'emp_length_num', 'grade_num', 'sub_grade_num', 'delinq_2yrs_zero', 'pub_rec_zero', 'collections_12_mths_zero', 'short_emp', 'payment_in

In [25]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : -1 if x ==1 else 1)

In [26]:
loans = loans.drop('bad_loans',axis=1)
loans.head(5)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none,safe_loans
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1,-1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1,1


In [27]:
print "Safe loans",len(loans[loans['safe_loans'] == 1])/float(len(loans))
print "Risky loans",len(loans[loans['safe_loans'] == -1])/float(len(loans))

Safe loans 0.811185331996
Risky loans 0.188814668004


In [28]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'                    # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
loans = loans[features + [target]]

In [29]:
data_train_id = pd.read_json('../data/module-5-assignment-1-train-idx.json')
data_validation_id = pd.read_json('../data/module-5-assignment-1-validation-idx.json')

In [30]:
data_train = loans.iloc[data_train_id]
data_validation = loans.iloc[data_validation_id]

In [31]:
def get_numpy_data(data_frame,features,classifier):
    feature_matrix = data_frame[features].as_matrix()
    classifier_matrix = data_frame[classifier].as_matrix()
    return feature_matrix,classifier_matrix

In [32]:
feature_matrix_train,target_matrix_train = get_numpy_data(loans,features,target)

In [33]:
print "feature_matrix_train : ",feature_matrix_train.shape
print "target_matrix_train : ",target_matrix_train.shape

feature_matrix_train :  (122607, 12)
target_matrix_train :  (122607,)


In [34]:
from sklearn import tree

In [35]:
decision_tree_model = tree.DecisionTreeClassifier(max_depth=6)
decision_tree_model.fit(feature_matrix_train,target_matrix_train)

ValueError: invalid literal for float(): 36 months

In [43]:
for x in features:
    print x,len(loans[x].unique())

grade 7
sub_grade 35
short_emp 2
emp_length_num 12
home_ownership 4
dti 3543
purpose 12
term 2
last_delinq_none 2
last_major_derog_none 2
revol_util 1156
total_rec_late_fee 2472


In [56]:
print loans.dtypes
strings = list(loans.select_dtypes(include=['object']))

grade                     object
sub_grade                 object
short_emp                  int64
emp_length_num             int64
home_ownership            object
dti                      float64
purpose                   object
term                      object
last_delinq_none           int64
last_major_derog_none      int64
revol_util               float64
total_rec_late_fee       float64
safe_loans                 int64
dtype: object


In [57]:
for x in strings:
    print len(loans[x].unique())

7
35
4
12
2
